In [1]:
import os

import matplotlib.pyplot as plt
import tensorflow as tf

from src.dataset import *
from src.model import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Write TFRecord shards

In [2]:
# TFRWriter(image_size=256).write()

# Initialize model

In [3]:
config = {
    'local_depth': 8,
    'local_convdepth': 16,
    'local_imagesize': 256,
    'global_depth': 5,
    'global_convdepth': 32,
    'global_imagesize': 128,
    'scale': 0.25}

model = CIEXYZNet(**config)

2022-06-06 02:37:41.297644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-06 02:37:41.324027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-06 02:37:41.324240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:938] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-06 02:37:41.324545: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with t

# Build model

In [4]:
dataloader = DataLoader('testing', image_size=256)
data = dataloader.data

In [5]:
for i, batch in enumerate(data):
    logits = model(batch['sRGB_image'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,10))

idx = 0
print("Filename:", batch['filename'][idx].numpy().decode())
ax[0].imshow(batch['sRGB_image'][idx])
ax[1].imshow(batch['XYZ_image'][idx])
ax[0].set_title("sRGB")
ax[1].set_title("XYZ")
ax[0].axis("off")
ax[1].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.srgb2xyz_localnet.summary()

In [ ]:
model.srgb2xyz_globalnet.summary()

In [ ]:
model.xyz2srgb_localnet.summary()

In [ ]:
model.xyz2srgb_globalnet.summary()